In [1]:
from flask import Flask, redirect, request, session, url_for, jsonify
from google_auth_oauthlib.flow import Flow
from googleapiclient.discovery import build
import os
from datetime import datetime, timedelta
import uuid

In [2]:
app = Flask(__name__)
app.secret_key = 'tu_clave_secreta_aqui'  # Cambia esto en producción

# Configuración de OAuth
CLIENT_SECRETS_FILE = "client_secret.json"  # Descargado de Google Cloud Console
SCOPES = ['https://www.googleapis.com/auth/drive.file']
API_SERVICE_NAME = 'drive'
API_VERSION = 'v3'

# Ruta temporal para almacenar credenciales (en producción usa una base de datos)
credentials_store = {}

@app.route('/')
def index():
    return "Bienvenido a la aplicación de Google Drive"

@app.route('/auth')
def auth():
    # Generar un estado único para prevenir CSRF
    state = str(uuid.uuid4())
    session['state'] = state
    
    # Crear flujo de autenticación
    flow = Flow.from_client_secrets_file(
        CLIENT_SECRETS_FILE,
        scopes=SCOPES,
        state=state
    )
    
    # URL de redireccionamiento debe coincidir con la configurada en Google Cloud
    flow.redirect_uri = url_for('oauth_callback', _external=True)
    
    authorization_url, _ = flow.authorization_url(
        access_type='offline',
        include_granted_scopes='true'
    )
    
    return redirect(authorization_url)

@app.route('/auth/callback')
def oauth_callback():
    # Verificar estado para prevenir CSRF
    if session.get('state') != request.args.get('state'):
        return 'Estado no válido', 400
    
    flow = Flow.from_client_secrets_file(
        CLIENT_SECRETS_FILE,
        scopes=SCOPES,
        state=session['state']
    )
    flow.redirect_uri = url_for('oauth_callback', _external=True)
    
    # Obtener token de acceso
    flow.fetch_token(authorization_response=request.url)
    credentials = flow.credentials
    
    # Almacenar credenciales (en producción usa una base de datos)
    user_id = str(uuid.uuid4())  # Identificador único del usuario
    credentials_store[user_id] = {
        'token': credentials.token,
        'refresh_token': credentials.refresh_token,
        'token_uri': credentials.token_uri,
        'client_id': credentials.client_id,
        'client_secret': credentials.client_secret,
        'scopes': credentials.scopes,
        'expiry': credentials.expiry.isoformat() if credentials.expiry else None
    }
    
    # Redirigir al frontend con el user_id
    return redirect(f"http://tufrontend.com?user_id={user_id}")

@app.route('/upload', methods=['POST'])
def upload_file():
    user_id = request.json.get('user_id')
    if not user_id or user_id not in credentials_store:
        return jsonify({'error': 'Usuario no autenticado'}), 401
    
    # Reconstruir credenciales
    creds_data = credentials_store[user_id]
    creds = Credentials(
        token=creds_data['token'],
        refresh_token=creds_data['refresh_token'],
        token_uri=creds_data['token_uri'],
        client_id=creds_data['client_id'],
        client_secret=creds_data['client_secret'],
        scopes=creds_data['scopes'],
        expiry=datetime.fromisoformat(creds_data['expiry']) if creds_data['expiry'] else None
    )
    
    # Crear servicio de Google Drive
    service = build(API_SERVICE_NAME, API_VERSION, credentials=creds)
    
    # Subir archivo (ejemplo con imagen en base64)
    file_metadata = {
        'name': 'imagen_semanal.png',
        'mimeType': 'image/png'
    }
    
    # Aquí procesarías la imagen del request
    # Ejemplo simplificado:
    media = MediaFileUpload('ruta/a/tu/imagen.png', mimetype='image/png')
    
    file = service.files().create(
        body=file_metadata,
        media_body=media,
        fields='id'
    ).execute()
    
    return jsonify({'file_id': file.get('id')})


In [ ]:

if __name__ == '__main__':
    os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'  # Solo para desarrollo
    app.run(port=8000)

 * Tip: There are .env files present. Install python-dotenv to use them.


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8000
Press CTRL+C to quit
[2025-05-23 15:43:54,318] ERROR in app: Exception on /auth [GET]
Traceback (most recent call last):
  File "/home/hi4i/facebook/backup_social_network/.venv/lib/python3.12/site-packages/flask/app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/hi4i/facebook/backup_social_network/.venv/lib/python3.12/site-packages/flask/app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/hi4i/facebook/backup_social_network/.venv/lib/python3.12/site-packages/flask/app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/hi4i/facebook/backup_social_network/.venv/lib/python3.12/site-packages/flask/app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ig